In [48]:
import torch
from torch import nn,save,load
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm

In [3]:
device='cpu'
if torch.cuda.is_available():
    device='cuda'

In [7]:
train = datasets.MNIST(root="data",download=True,train=True,transform=ToTensor())

100%|██████████| 9912422/9912422 [00:20<00:00, 485722.60it/s] 


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 831993.06it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:03<00:00, 493188.42it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 309286.94it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [10]:
dataset = DataLoader(train,32)

In [36]:
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1,32,(3,3)),
            nn.ReLU(),
            nn.Conv2d(32,64,(3,3)),
            nn.ReLU(),
            nn.Conv2d(64,64,(3,3)),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64*(28-6)*(28-6),10)
        )
    def forward(self,x):
        return self.model(x)

    
model = ImageClassifier().to('cpu')

In [38]:
opt = Adam(model.parameters(),lr=1e-3)
loss = nn.CrossEntropyLoss()

In [42]:
for epoch in range(10):
    for batch in tqdm(dataset):
        x,y = batch
        x,y  = x.to('cpu'),y.to('cpu')

        yhat = model(x)
        loss_fn = loss(yhat,y)

        opt.zero_grad()
        loss_fn.backward()
        opt.step()
    print(f"Ephoch => {epoch} loss => {loss_fn} ")

100%|██████████| 1875/1875 [01:44<00:00, 17.97it/s]


Ephoch => 0 loss => 0.0036432999186217785 


100%|██████████| 1875/1875 [01:42<00:00, 18.36it/s]


Ephoch => 1 loss => 0.0001731215015752241 


100%|██████████| 1875/1875 [01:40<00:00, 18.74it/s]


...

Ephoch => 7 loss => 0.0005602602614089847 


100%|██████████| 1875/1875 [01:43<00:00, 18.14it/s]


Ephoch => 8 loss => 8.068202987487894e-06 


100%|██████████| 1875/1875 [01:51<00:00, 16.85it/s]

Ephoch => 9 loss => 4.413982242112979e-05 


In [54]:
import cv2
image = cv2.imread("img_1.jpg",0)

image = ToTensor()(image).unsqueeze(0).to('cpu')
image.shape

torch.Size([1, 1, 28, 28])

In [56]:
torch.argmax(model(image))

tensor(2)

In [57]:
with open('model_state.pt','wb') as f:
    save(model.state_dict(),f)

In [58]:
with open('model_state.pt','rb') as f:
    model.load_state_dict(load(f))

In [60]:
model

ImageClassifier(
  (model): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=30976, out_features=10, bias=True)
  )
)